Copyright (c) Microsoft Corporation.

Licensed under the MIT License.

# HRNet training and validation on numpy dataset

In this notebook, we demonstrate how to train an HRNet model for facies prediction using [Penobscot](https://zenodo.org/record/1341774#.XepaaUB2vOg) dataset. The Penobscot 3D seismic dataset was acquired in the Scotian shelf, offshore Nova Scotia, Canada. Please refer to the top-level [README.md](../../../README.md) file to download and prepare this dataset for the experiments. 

The data expected in this notebook needs to be in the form of two 3D numpy arrays. One array will contain the seismic information, the other the mask. The network will be trained to take a 2D patch of data from the seismic block and learn to predict the 2D mask patch associated with it.

## Environment setup

To set up the conda environment, please follow the instructions in the top-level [README.md](../../../README.md) file.

__Note__: To register the conda environment in Jupyter, run:
`python -m ipykernel install --user --name envname`


## Library imports

In [ ]:
import logging
import logging.config
from os import path

import cv2
import numpy as np
import yacs.config
import torch
from albumentations import Compose, HorizontalFlip, Normalize, PadIfNeeded, Resize
from cv_lib.utils import load_log_configuration
from cv_lib.event_handlers import (
    SnapshotHandler,
    logging_handlers,
    tensorboard_handlers,
)
from cv_lib.event_handlers.logging_handlers import Evaluator
from cv_lib.event_handlers.tensorboard_handlers import (
    create_image_writer,
    create_summary_writer,
)
from cv_lib.segmentation import models, extract_metric_from
from cv_lib.segmentation.metrics import (
    pixelwise_accuracy,
    class_accuracy,
    mean_class_accuracy,
    class_iou,
    mean_iou,
)
from cv_lib.segmentation.dutchf3.utils import (
    current_datetime,
    generate_path,
    np_to_tb,
)
from cv_lib.segmentation.penobscot.engine import (
    create_supervised_evaluator,
    create_supervised_trainer,
)
from deepseismic_interpretation.penobscot.data import PenobscotInlinePatchDataset
from deepseismic_interpretation.dutchf3.data import decode_segmap
from ignite.contrib.handlers import CosineAnnealingScheduler
from ignite.engine import Events
from ignite.metrics import Loss
from ignite.utils import convert_tensor
from toolz import compose
from torch.utils import data
from itkwidgets import view
from utilities import plot_aline
from toolz import take


mask_value = 255
_SEG_COLOURS = np.asarray(
    [[241, 238, 246], [208, 209, 230], [166, 189, 219], [116, 169, 207], [54, 144, 192], [5, 112, 176], [3, 78, 123]]
)

# experiment configuration file
CONFIG_FILE = "./configs/hrnet.yaml"

In [ ]:
def _prepare_batch(batch, device=None, non_blocking=False):
    x, y, ids, patch_locations = batch
    return (
        convert_tensor(x, device=device, non_blocking=non_blocking),
        convert_tensor(y, device=device, non_blocking=non_blocking),
        ids,
        patch_locations,
    )

## Experiment configuration file
We use configuration files to specify experiment configuration, such as hyperparameters used in training and evaluation, as well as other experiment settings. We provide several configuration files for this notebook, under `./configs`, mainly differing in the DNN architecture used for defining the model.

Modify the `CONFIG_FILE` variable above if you would like to run the experiment using a different configuration file.

In [ ]:
with open(CONFIG_FILE, "rt") as f_read:
    config = yacs.config.load_cfg(f_read)

print(f'Configuration loaded. Please check that the DATASET.ROOT:{config.DATASET.ROOT} points to your data location.')
print(f'To modify any of the options, please edit the configuration file {CONFIG_FILE} and reload. \n')
print(config)

## Parameters

In [ ]:
# The number of datapoints you want to run in training or validation per batch 
# Setting to None will run whole dataset
# useful for integration tests with a setting of something like 3
# Use only if you want to check things are running and don't want to run
# through whole dataset
max_iterations = None  
# The number of epochs to run in training
max_epochs = config.TRAIN.END_EPOCH  
max_snapshots = config.TRAIN.SNAPSHOTS
dataset_root = config.DATASET.ROOT

## Load Dataset

In [ ]:
import os
from toolz import pipe
import glob
from PIL import Image

In [ ]:
image_dir = os.path.join(dataset_root, "inlines")
mask_dir = os.path.join(dataset_root, "masks")

image_iter = pipe(os.path.join(image_dir, "*.tiff"), glob.iglob,)

_open_to_array = compose(np.array, Image.open)


def open_image_mask(image_path):
    return pipe(image_path, _open_to_array)


def _mask_filename(imagepath):
    file_part = os.path.splitext(os.path.split(imagepath)[-1].strip())[0]
    return os.path.join(mask_dir, file_part + "_mask.png")


image_list = sorted(list(image_iter))
image_list_array = [_open_to_array(i) for i in image_list]
mask_list_array = [pipe(i, _mask_filename, _open_to_array) for i in image_list]
mask = np.stack(mask_list_array, axis=0)

Let's visualize the dataset.

In [ ]:
view(mask, slicing_planes=True)

Let's view slices of the data along inline and crossline directions.

In [ ]:
idx = 100
x_in = image_list_array[idx]
x_inl = mask_list_array[idx]

plot_aline(x_in, x_inl, xlabel="inline")

## Model training

In [ ]:
# Setup logging
load_log_configuration(config.LOG_CONFIG)
logger = logging.getLogger(__name__)
logger.debug(config.WORKERS)
scheduler_step = max_epochs // max_snapshots
torch.backends.cudnn.benchmark = config.CUDNN.BENCHMARK

torch.manual_seed(config.SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(config.SEED)
np.random.seed(seed=config.SEED)

### Set up data augmentation

Let's define our data augmentation pipeline, which includes basic transformations, such as _data normalization, resizing, and padding_ if necessary.
The padding is carried out twice becuase if we split the inline or crossline slice into multiple patches then some of these patches will be at the edge of the slice and may not contain a full patch worth of data. To compensate to this and have same size patches in the batch (a requirement) we need to pad them.
So our basic augmentation is:
- Normalize
- Pad if needed to initial size
- Resize to a larger size
- Pad further if necessary

In [ ]:
# Setup Augmentations
basic_aug = Compose(
    [
        Normalize(mean=(config.TRAIN.MEAN,), std=(config.TRAIN.STD,), max_pixel_value=config.TRAIN.MAX,),
        PadIfNeeded(
            min_height=config.TRAIN.PATCH_SIZE,
            min_width=config.TRAIN.PATCH_SIZE,
            border_mode=cv2.BORDER_CONSTANT,
            always_apply=True,
            mask_value=mask_value,
            value=0,
        ),
        Resize(config.TRAIN.AUGMENTATIONS.RESIZE.HEIGHT, config.TRAIN.AUGMENTATIONS.RESIZE.WIDTH, always_apply=True,),
        PadIfNeeded(
            min_height=config.TRAIN.AUGMENTATIONS.PAD.HEIGHT,
            min_width=config.TRAIN.AUGMENTATIONS.PAD.WIDTH,
            border_mode=cv2.BORDER_CONSTANT,
            always_apply=True,
            mask_value=mask_value,
            value=0,
        ),
    ]
)
if config.TRAIN.AUGMENTATION:
    train_aug = Compose([basic_aug, HorizontalFlip(p=0.5)])
    val_aug = basic_aug
else:
    train_aug = val_aug = basic_aug

### Load the data

For training the model, we will use a patch-based approach. Rather than using entire sections (crosslines or inlines) of the data, we extract a large number of small patches from the sections, and use the patches as our data. This allows us to generate larger set of images for training, but is also a more feasible approach for large seismic volumes.

We are using a custom patch data loader from our __`deepseismic_interpretation`__ library for generating and loading patches from seismic section data.

In [ ]:
train_set = PenobscotInlinePatchDataset(
    dataset_root,
    config.TRAIN.PATCH_SIZE,
    config.TRAIN.STRIDE,
    split="train",
    transforms=train_aug,
    n_channels=config.MODEL.IN_CHANNELS,
    complete_patches_only=config.TRAIN.COMPLETE_PATCHES_ONLY,
)

val_set = PenobscotInlinePatchDataset(
    dataset_root,
    config.TRAIN.PATCH_SIZE,
    config.TRAIN.STRIDE,
    split="val",
    transforms=val_aug,
    n_channels=config.MODEL.IN_CHANNELS,
    complete_patches_only=config.VALIDATION.COMPLETE_PATCHES_ONLY,
)

logger.info(train_set)
logger.info(val_set)

n_classes = train_set.n_classes
train_loader = data.DataLoader(
    train_set, batch_size=config.TRAIN.BATCH_SIZE_PER_GPU, num_workers=config.WORKERS, shuffle=True,
)

val_loader = data.DataLoader(val_set, batch_size=config.VALIDATION.BATCH_SIZE_PER_GPU, num_workers=config.WORKERS,)

### Set up model training
Next, let's define a model to train, an optimization algorithm, and a loss function.

Note that the model is loaded from our __`cv_lib`__ library, using the name of the model as specified in the configuration file. To load a different model, either change the `MODEL.NAME` field in the configuration file, or create a new one corresponding to the model you wish to train.

In [ ]:
model = getattr(models, config.MODEL.NAME).get_seg_model(config)

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
model = model.to(device)  # Send to GPU

optimizer = torch.optim.SGD(
    model.parameters(), lr=config.TRAIN.MAX_LR, momentum=config.TRAIN.MOMENTUM, weight_decay=config.TRAIN.WEIGHT_DECAY,
)

output_dir = generate_path(config.OUTPUT_DIR, config.MODEL.NAME, current_datetime(),)
summary_writer = create_summary_writer(log_dir=path.join(output_dir, config.LOG_DIR))
snapshot_duration = scheduler_step * len(train_loader)
scheduler = CosineAnnealingScheduler(optimizer, "lr", config.TRAIN.MAX_LR, config.TRAIN.MIN_LR, snapshot_duration)

criterion = torch.nn.CrossEntropyLoss(ignore_index=mask_value, reduction="mean")

### Training the model
We use [ignite](https://pytorch.org/ignite/index.html) framework to create training and validation loops in our codebase. Ignite provides an easy way to create compact training/validation loops without too much boilerplate code.

In this notebook, we demonstrate the use of ignite on the training loop only. We create a training engine `trainer` that loops multiple times over the training dataset and updates model parameters. In addition, we add various events to the trainer, using an event system, that allows us to interact with the engine on each step of the run, such as, when the trainer is started/completed, when the epoch is started/completed and so on.

In the cell below, we use event handlers to add the following events to the training loop:
- log training output
- log and schedule learning rate and
- periodically save model to disk.

In [ ]:
trainer = create_supervised_trainer(model, optimizer, criterion, _prepare_batch, device=device)

trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)

trainer.add_event_handler(
    Events.ITERATION_COMPLETED, logging_handlers.log_training_output(log_interval=config.PRINT_FREQ),
)
trainer.add_event_handler(Events.EPOCH_STARTED, logging_handlers.log_lr(optimizer))
trainer.add_event_handler(
    Events.EPOCH_STARTED, tensorboard_handlers.log_lr(summary_writer, optimizer, "epoch"),
)
trainer.add_event_handler(
    Events.ITERATION_COMPLETED, tensorboard_handlers.log_training_output(summary_writer),
)

In [ ]:
def _select_pred_and_mask(model_out_dict):
    return (model_out_dict["y_pred"].squeeze(), model_out_dict["mask"].squeeze())


evaluator = create_supervised_evaluator(
    model,
    _prepare_batch,
    metrics={
        "pixacc": pixelwise_accuracy(n_classes, output_transform=_select_pred_and_mask),
        "nll": Loss(criterion, output_transform=_select_pred_and_mask),
        "cacc": class_accuracy(n_classes, output_transform=_select_pred_and_mask),
        "mca": mean_class_accuracy(n_classes, output_transform=_select_pred_and_mask),
        "ciou": class_iou(n_classes, output_transform=_select_pred_and_mask),
        "mIoU": mean_iou(n_classes, output_transform=_select_pred_and_mask),
    },
    device=device,
)

if max_iterations is not None:
    val_loader = take(max_iterations, val_loader)

# Set the validation run to start on the epoch completion of the training run
trainer.add_event_handler(Events.EPOCH_COMPLETED, Evaluator(evaluator, val_loader))

evaluator.add_event_handler(
    Events.EPOCH_COMPLETED,
    logging_handlers.log_metrics(
        "Validation results",
        metrics_dict={
            "nll": "Avg loss :",
            "pixacc": "Pixelwise Accuracy :",
            "mca": "Avg Class Accuracy :",
            "mIoU": "Avg Class IoU :",
        },
    ),
)
evaluator.add_event_handler(
    Events.EPOCH_COMPLETED,
    tensorboard_handlers.log_metrics(
        summary_writer,
        trainer,
        "epoch",
        metrics_dict={
            "mIoU": "Validation/mIoU",
            "nll": "Validation/Loss",
            "mca": "Validation/MCA",
            "pixacc": "Validation/Pixel_Acc",
        },
    ),
)


def _select_max(pred_tensor):
    return pred_tensor.max(1)[1]


def _tensor_to_numpy(pred_tensor):
    return pred_tensor.squeeze().cpu().numpy()


transform_func = compose(np_to_tb, decode_segmap(n_classes=n_classes, label_colours=_SEG_COLOURS), _tensor_to_numpy,)

transform_pred = compose(transform_func, _select_max)

evaluator.add_event_handler(
    Events.EPOCH_COMPLETED, create_image_writer(summary_writer, "Validation/Image", "image"),
)
evaluator.add_event_handler(
    Events.EPOCH_COMPLETED,
    create_image_writer(summary_writer, "Validation/Mask", "mask", transform_func=transform_func),
)
evaluator.add_event_handler(
    Events.EPOCH_COMPLETED,
    create_image_writer(summary_writer, "Validation/Pred", "y_pred", transform_func=transform_pred),
)

### Checkpointing
Below we define the function that will save the best performing models based on mean IoU.

In [ ]:
def snapshot_function():
    return (trainer.state.iteration % snapshot_duration) == 0


checkpoint_handler = SnapshotHandler(
    path.join(output_dir, config.TRAIN.MODEL_DIR), config.MODEL.NAME, extract_metric_from("mIoU"), snapshot_function,
)
evaluator.add_event_handler(Events.EPOCH_COMPLETED, checkpoint_handler, {"model": model})

Start the training engine run.

In [ ]:
if max_iterations is not None:
    train_loader = take(max_iterations, train_loader)

In [ ]:
logger.info("Starting training")
trainer.run(train_loader, max_epochs=max_epochs)

## Tensorboard
Using tensorboard for monitoring runs can be quite enlightening. Just ensure that the appropriate port is open on the VM so you can access it. Below we have the command for running tensorboard in your notebook. You can as easily view it in a seperate browser window by pointing the browser to the appropriate location and port.

In [ ]:
if max_epochs>1:
    %load_ext tensorboard

In [ ]:
if max_epochs>1:
    %tensorboard --logdir outputs --port 6007 --host 0.0.0.0